<a href="https://colab.research.google.com/github/yukinaga/ai_music/blob/main/section_2/02_simple_music_vae.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Music VAE를 사용해 보자!
「Music VAE」를 사용해서 두 노트 Sequence 사이를 보간하도록 곡을 생성합시다.

## 라이브러리 설치
Magenta와 함께 음악 생성용 라이브러리 pyFluid Synth, MIDI 데이터를 처리하기 위한 pretty_midi 등을 설치합니다.

In [ ]:
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi
!pip install -qU magenta

## 체크포인트다운로드
학습된 모델의 다양한 데이터가 포함된 '체크포인트' 파일을 다운로드 받아 '/content/'에 저장합니다.  

In [ ]:
!gsutil -q -m cp -R gs://download.magenta.tensorflow.org/models/music_vae/colab2/checkpoints/mel_2bar_big.ckpt.* /content/

## 모델의 초기화
체크 포인트를 읽어 Music VAE의 훈련이 끝난 모델을 초기화합니다.  
Music VAEの設定: https://github.com/magenta/magenta/tree/main/magenta/models/music_vae

In [ ]:
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel

# 모델의 초기화
music_vae = TrainedModel(
      configs.CONFIG_MAP["cat-mel_2bar_big"], 
      batch_size=4,  # 一度に処理するデータ数
      checkpoint_dir_or_path="/content/mel_2bar_big.ckpt")

## NoteSequenceの生成

Music VAE를 통해 Notequence를 생성합니다.  
이 때, 생성하는 NoteSequence의 수, 스텝 수, 온도(랜덤 정도)를 설정할 수 있습니다.  

In [ ]:
import note_seq

generated = music_vae.sample(n=5,  # 生成数
                             length=128,  # ステップ数
                             temperature=1.0)  # 温度

for ns in generated:
    note_seq.plot_sequence(ns)
    note_seq.play_sequence(ns, synth=note_seq.fluidsynth)

## 처음과 마지막 노트 Sequence

곡의 처음과 마지막 노트 Sequence를 설정하고, 그 사이를 Music VAE로 보간합니다.  


In [ ]:
import magenta
import note_seq
from note_seq.protobuf import music_pb2

# 最初のNoteSeqence
kira2_start = music_pb2.NoteSequence()

kira2_start.notes.add(pitch=60, start_time=0.0, end_time=0.4, velocity=80)
kira2_start.notes.add(pitch=60, start_time=0.4, end_time=0.8, velocity=80)
kira2_start.notes.add(pitch=67, start_time=0.8, end_time=1.2, velocity=80)
kira2_start.notes.add(pitch=67, start_time=1.2, end_time=1.6, velocity=80)
kira2_start.notes.add(pitch=69, start_time=1.6, end_time=2.0, velocity=80)
kira2_start.notes.add(pitch=69, start_time=2.0, end_time=2.4, velocity=80)
kira2_start.notes.add(pitch=67, start_time=2.4, end_time=3.2, velocity=80)
kira2_start.notes.add(pitch=65, start_time=3.2, end_time=3.6, velocity=80)
kira2_start.notes.add(pitch=65, start_time=3.6, end_time=4.0, velocity=80)
kira2_start.notes.add(pitch=64, start_time=4.0, end_time=4.4, velocity=80)
kira2_start.notes.add(pitch=64, start_time=4.4, end_time=4.8, velocity=80)
kira2_start.notes.add(pitch=62, start_time=4.8, end_time=5.2, velocity=80)
kira2_start.notes.add(pitch=62, start_time=5.2, end_time=5.6, velocity=80)
kira2_start.notes.add(pitch=60, start_time=5.6, end_time=6.4, velocity=80) 

kira2_start.total_time = 6.4 
kira2_start.tempos.add(qpm=75);

note_seq.plot_sequence(kira2_start)
note_seq.play_sequence(kira2_start, synth=note_seq.fluidsynth)

# 最後のNoteSeqence
kira2_end = music_pb2.NoteSequence()

kira2_end.notes.add(pitch=60, start_time=0.0, end_time=0.4, velocity=80)
kira2_end.notes.add(pitch=62, start_time=0.4, end_time=0.8, velocity=80)
kira2_end.notes.add(pitch=64, start_time=0.8, end_time=1.2, velocity=80)
kira2_end.notes.add(pitch=67, start_time=1.2, end_time=1.6, velocity=80)
kira2_end.notes.add(pitch=69, start_time=1.6, end_time=2.0, velocity=80)
kira2_end.notes.add(pitch=64, start_time=2.0, end_time=2.4, velocity=80)
kira2_end.notes.add(pitch=60, start_time=2.4, end_time=3.2, velocity=80)
kira2_end.notes.add(pitch=62, start_time=3.2, end_time=3.6, velocity=80)
kira2_end.notes.add(pitch=64, start_time=3.6, end_time=4.0, velocity=80)
kira2_end.notes.add(pitch=67, start_time=4.0, end_time=4.4, velocity=80)
kira2_end.notes.add(pitch=69, start_time=4.4, end_time=4.8, velocity=80)
kira2_end.notes.add(pitch=64, start_time=4.8, end_time=5.2, velocity=80)
kira2_end.notes.add(pitch=62, start_time=5.2, end_time=5.6, velocity=80)
kira2_end.notes.add(pitch=60, start_time=5.6, end_time=6.4, velocity=80) 

kira2_end.total_time = 6.4
kira2_end.tempos.add(qpm=75); 

note_seq.plot_sequence(kira2_end)
note_seq.play_sequence(kira2_end, synth=note_seq.fluidsynth)  # NoteSequenceの再生

## 노트Sequnce간의 보간

`interpolate` 메서드를 통해 두 NoteSequence 사이를 보간하도록 곡을 생성합니다.

In [ ]:
n_seq=8#곡의 노트Seqence 수 (처음과 끝을 포함) 

# NoteSeqenceを複数生成し、リストに格納
gen_seq = music_vae.interpolate(
    kira2_start,  # 最初のNoteSeqence
    kira2_end,  # 最後のNoteSeqence
    num_steps=n_seq,
    length=32)

# NoteSeqenceを全て結合し、1つの曲に
interp_seq = note_seq.sequences_lib.concatenate_sequences(gen_seq)

note_seq.plot_sequence(interp_seq)
note_seq.play_sequence(interp_seq, synth=note_seq.fluidsynth)

## MIDI 파일 저장 및 다운로드
`노트Sequence`를 MIDI 데이터로 변환하여 저장하고 다운로드합니다.


In [ ]:
from google.colab import files

note_seq.sequence_proto_to_midi_file(interp_seq, "simple_music_vae.mid")  #MIDI　データに変換し保存
files.download("simple_music_vae.mid")  # ダウンロード